<a href="https://colab.research.google.com/github/mocci24/RVCGI-V2/blob/main/RVCV2_Tools_Mocci.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RVC TRAINING V2 ALL IN ONE

DON'T RUN ALL THE RUNTIME!!

CHOSE WHAT YOU WANT

TRAIN, SPLITTING, DOWNLOADING AUDIO, SEPARATE VOCAL, OR INFERENCE

# 1. Preparation

In [ ]:
#@title Test Runtime
!nvidia-smi
!nvcc -V
!free -h

In [ ]:
#@title Mount Drive

from google.colab import drive
drive.mount('/content/drive')

# 2. SELECT MODE

In [ ]:
#@title Input Form

Mode = "training" #@param ["Separate", "Splitting", "training", "inference"]
dataset = "Youtube" #@param ["Youtube", "Drive"]
url = "" #@param {type:"string"}
drive_path = "" #@param {type:"string"}
AUDIO_NAME = "" #@param {type:"string"}
#@markdown fill modelname if training/resume training
MODELNAME = "kazuha"  #@param {type:"string"}


# 3. This option for download wav audio from YOUTUBE

In [ ]:
#@title Install Library for Youtube WAV Download
if dataset == "Drive":
  print("Dataset is set to Drive. Skipping this section")
elif dataset == "Youtube":
  !pip install yt_dlp
  !pip install ffmpeg
  !mkdir youtubeaudio

In [ ]:
#@title Download Youtube WAV
from __future__ import unicode_literals

if dataset == "Drive":
  print("Dataset is set to Drive. Skipping this section")
elif dataset == "Youtube":
  import yt_dlp
  import ffmpeg
  import sys


  ydl_opts = {
      'format': 'bestaudio/best',
  #    'outtmpl': 'output.%(ext)s',
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'wav',
      }],
      "outtmpl": f'youtubeaudio/{AUDIO_NAME}',  # this is where you can edit how you'd like the filenames to be formatted
  }
  def download_from_url(url):
      ydl.download([url])
      # stream = ffmpeg.input('output.m4a')
      # stream = ffmpeg.output(stream, 'output.wav')


  with yt_dlp.YoutubeDL(ydl_opts) as ydl:

        download_from_url(url)

# 4. Separating vocal & instrument with demucs

In [ ]:
#@title Install Demucs for Separating Audio
!python3 -m pip install -U demucs

In [ ]:
#@title Start Separate Vocal and Instrument using Demucs
#@markdown This program is standart separating Vocal and Instrument.
#@markdown Use webUI for more feature.
import subprocess
input_file = "" #@param {type:"string"}
OUTPUT_DIR = "/content/separated"

command = f"demucs --two-stems=vocals {input_file}"
result = subprocess.run(command.split(), stdout=subprocess.PIPE)
print(result.stdout.decode())
#input_file = input("Masukkan file audio: ")

# 5. Split audio to 10-sec segments for dataset sample before training

In [ ]:
#@title Split The Audio into Smaller Duration Before Training
if Mode == "Separate":
    print("Mode is set to Separate. Skipping this section")
elif Mode ==  "Splitting":
  !pip install numpy
  !pip install librosa
  !pip install soundfile
  !mkdir -p dataset/{AUDIO_NAME}

In [ ]:
#@title
import numpy as np
import librosa
import soundfile


# This function is obtained from librosa.
def get_rms(
    y,
    *,
    frame_length=2048,
    hop_length=512,
    pad_mode="constant",
):
    padding = (int(frame_length // 2), int(frame_length // 2))
    y = np.pad(y, padding, mode=pad_mode)

    axis = -1
    # put our new within-frame axis at the end for now
    out_strides = y.strides + tuple([y.strides[axis]])
    # Reduce the shape on the framing axis
    x_shape_trimmed = list(y.shape)
    x_shape_trimmed[axis] -= frame_length - 1
    out_shape = tuple(x_shape_trimmed) + tuple([frame_length])
    xw = np.lib.stride_tricks.as_strided(
        y, shape=out_shape, strides=out_strides
    )
    if axis < 0:
        target_axis = axis - 1
    else:
        target_axis = axis + 1
    xw = np.moveaxis(xw, -1, target_axis)
    # Downsample along the target axis
    slices = [slice(None)] * xw.ndim
    slices[axis] = slice(0, None, hop_length)
    x = xw[tuple(slices)]

    # Calculate power
    power = np.mean(np.abs(x) ** 2, axis=-2, keepdims=True)

    return np.sqrt(power)


class Slicer:
    def __init__(self,
                 sr: int,
                 threshold: float = -40.,
                 min_length: int = 5000,
                 min_interval: int = 300,
                 hop_size: int = 20,
                 max_sil_kept: int = 5000):
        if not min_length >= min_interval >= hop_size:
            raise ValueError('The following condition must be satisfied: min_length >= min_interval >= hop_size')
        if not max_sil_kept >= hop_size:
            raise ValueError('The following condition must be satisfied: max_sil_kept >= hop_size')
        min_interval = sr * min_interval / 1000
        self.threshold = 10 ** (threshold / 20.)
        self.hop_size = round(sr * hop_size / 1000)
        self.win_size = min(round(min_interval), 4 * self.hop_size)
        self.min_length = round(sr * min_length / 1000 / self.hop_size)
        self.min_interval = round(min_interval / self.hop_size)
        self.max_sil_kept = round(sr * max_sil_kept / 1000 / self.hop_size)

    def _apply_slice(self, waveform, begin, end):
        if len(waveform.shape) > 1:
            return waveform[:, begin * self.hop_size: min(waveform.shape[1], end * self.hop_size)]
        else:
            return waveform[begin * self.hop_size: min(waveform.shape[0], end * self.hop_size)]

    def slice(self, waveform):
        if len(waveform.shape) > 1:
            samples = waveform.mean(axis=0)
        else:
            samples = waveform
        if samples.shape[0] <= self.min_length:
            return [waveform]
        rms_list = get_rms(y=samples, frame_length=self.win_size, hop_length=self.hop_size).squeeze(0)
        sil_tags = []
        silence_start = None
        clip_start = 0
        for i, rms in enumerate(rms_list):
            # Keep looping while frame is silent.
            if rms < self.threshold:
                # Record start of silent frames.
                if silence_start is None:
                    silence_start = i
                continue
            # Keep looping while frame is not silent and silence start has not been recorded.
            if silence_start is None:
                continue
            # Clear recorded silence start if interval is not enough or clip is too short
            is_leading_silence = silence_start == 0 and i > self.max_sil_kept
            need_slice_middle = i - silence_start >= self.min_interval and i - clip_start >= self.min_length
            if not is_leading_silence and not need_slice_middle:
                silence_start = None
                continue
            # Need slicing. Record the range of silent frames to be removed.
            if i - silence_start <= self.max_sil_kept:
                pos = rms_list[silence_start: i + 1].argmin() + silence_start
                if silence_start == 0:
                    sil_tags.append((0, pos))
                else:
                    sil_tags.append((pos, pos))
                clip_start = pos
            elif i - silence_start <= self.max_sil_kept * 2:
                pos = rms_list[i - self.max_sil_kept: silence_start + self.max_sil_kept + 1].argmin()
                pos += i - self.max_sil_kept
                pos_l = rms_list[silence_start: silence_start + self.max_sil_kept + 1].argmin() + silence_start
                pos_r = rms_list[i - self.max_sil_kept: i + 1].argmin() + i - self.max_sil_kept
                if silence_start == 0:
                    sil_tags.append((0, pos_r))
                    clip_start = pos_r
                else:
                    sil_tags.append((min(pos_l, pos), max(pos_r, pos)))
                    clip_start = max(pos_r, pos)
            else:
                pos_l = rms_list[silence_start: silence_start + self.max_sil_kept + 1].argmin() + silence_start
                pos_r = rms_list[i - self.max_sil_kept: i + 1].argmin() + i - self.max_sil_kept
                if silence_start == 0:
                    sil_tags.append((0, pos_r))
                else:
                    sil_tags.append((pos_l, pos_r))
                clip_start = pos_r
            silence_start = None
        # Deal with trailing silence.
        total_frames = rms_list.shape[0]
        if silence_start is not None and total_frames - silence_start >= self.min_interval:
            silence_end = min(total_frames, silence_start + self.max_sil_kept)
            pos = rms_list[silence_start: silence_end + 1].argmin() + silence_start
            sil_tags.append((pos, total_frames + 1))
        # Apply and return slices.
        if len(sil_tags) == 0:
            return [waveform]
        else:
            chunks = []
            if sil_tags[0][0] > 0:
                chunks.append(self._apply_slice(waveform, 0, sil_tags[0][0]))
            for i in range(len(sil_tags) - 1):
                chunks.append(self._apply_slice(waveform, sil_tags[i][1], sil_tags[i + 1][0]))
            if sil_tags[-1][1] < total_frames:
                chunks.append(self._apply_slice(waveform, sil_tags[-1][1], total_frames))
            return chunks

if Mode == "Separate":
    print("Mode is set to Separate. Skipping this section")

elif Mode ==  "Splitting":
  audio, sr = librosa.load(f'/content/separated/htdemucs/{AUDIO_NAME}/vocals.wav', sr=None, mono=False)  # Load an audio file with librosa.
  slicer = Slicer(
      sr=sr,
      threshold=-40,
      min_length=5000,
      min_interval=200,
      hop_size=10,
      max_sil_kept=500
  )
  chunks = slicer.slice(audio)
  for i, chunk in enumerate(chunks):
      if len(chunk.shape) > 1:
          chunk = chunk.T  # Swap axes if the audio is stereo.
      soundfile.write(f'/content/dataset/{AUDIO_NAME}/split_{i}.wav', chunk, sr)  # Save sliced audio files with soundfile.

In [ ]:
#@title Save splitting result to drive
if Mode == "Separate":
    print("Mode is set to Separate. Skipping this section")
elif Mode ==  "Splitting":
  !mkdir -p /content/drive/MyDrive/dataset/{AUDIO_NAME}
  !cp -r /content/dataset/{AUDIO_NAME}/* /content/drive/MyDrive/dataset/{AUDIO_NAME}

# 6. Load our models characters from Hugingface (optional)

In [ ]:
#@title Load models from Hugingface
#@markdown Click [here](https://huggingface.co/mocci24/RVCV2-GI) for more info
!apt install git-lfs
!git lfs install
!git clone https://huggingface.co/mocci24/RVCV2-GI

In [ ]:
#@title Cloning Github | Hugingface
#@markdown Run this if you want to use our models from HugingFace.
#@markdown Note: Don't run both "Clone Github" !<br />
#@markdown Our Models info
#@markdown [Here](https://huggingface.co/mocci24/RVCV2-GI)


!git clone https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI.git
!cp -r /content/RVCV2-GI/RVC/weights/* /content/Retrieval-based-Voice-Conversion-WebUI/weights
!mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/logs
!cp -r /content/RVCV2-GI/RVC/* /content/Retrieval-based-Voice-Conversion-WebUI/logs
%cd /content/Retrieval-based-Voice-Conversion-WebUI
!mkdir -p pretrained uvr5_weights

# 7. Load your models from Drive

In [ ]:
#@title Cloning Github | Drive
#@markdown Run this if you want to use models from your Google Drive.
#@markdown Note: Don't run both "Clone Github" !

!git clone https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI.git
!cp -r /content/drive/MyDrive/RVC/weights/* /content/Retrieval-based-Voice-Conversion-WebUI/weights
!mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}
!cp -r /content/drive/MyDrive/RVC/{MODELNAME}/* /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}
%cd /content/Retrieval-based-Voice-Conversion-WebUI
!mkdir -p pretrained uvr5_weights

# 8. Download the necessary requirements for training, inference, and others.

In [ ]:
!pip install gradio==3.34.0
!pip install -r requirements.txt

In [ ]:
#@title Install aria2
!apt -y install -qq aria2

In [ ]:
#@title Download requirements for training
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o G48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained/f0G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained -o f0G48k.pth

#RVC V2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o f0G40k.pth

In [ ]:
#@title Download requirements for Audio Splitting
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP2-人声vocals+非人声instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP2-人声vocals+非人声instrumentals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP5-主旋律人声vocals+其他instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP5-主旋律人声vocals+其他instrumentals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/5_HP-Karaoke-UVR.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o 5_HP-Karaoke-UVR.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP2_all_vocals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP2_all_vocals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP3_all_vocals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP3_all_vocals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP5_only_main_vocal.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP5_only_main_vocal.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/VR-DeEchoAggressive.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o VR-DeEchoAggressive.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/VR-DeEchoDeReverb.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o VR-DeEchoDeReverb.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/VR-DeEchoNormal.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o VR-DeEchoNormal.pth

In [ ]:
#@title Download hubert_base
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt -d /content/Retrieval-based-Voice-Conversion-WebUI -o hubert_base.pt

# 9. Run this if you want to split vocals into separate one-minute segments.

In [ ]:
#@title Instal requirements for audio splitter ( 1 minute per split )
!pip install pydub

In [ ]:
#@title Start Splitter ( 1-minute segments )
#@markdown Input file path, for example
#@markdown "/content/separated/vocal.wav"
#@markdown
#@markdown - What is this for?<br />
#@markdown To split the vocal duration into one-minute segments.<br />
#@markdown - Why does it need to be split?<br />
#@markdown To avoid "connection timeout" messages during inference when using the Havert/Crepe mode, as the maximum duration that can be processed is only around one minute.<br />
#@markdown However, if you are using the PM mode, you can skip this process.
input_file = "" #@param {type:"string"}
from pydub import AudioSegment
import os

def split_audio_per_minute(input_file):
    output_directory = '/content/Split'
    os.makedirs(output_directory, exist_ok=True)
    audio = AudioSegment.from_file(input_file)
    duration = len(audio)
    total_minutes = int(duration / (60 * 1000))

    for minute in range(total_minutes + 1):
        start_time = minute * 60 * 1000
        end_time = (minute + 1) * 60 * 1000
        split_audio = audio[start_time:end_time]
        output_file = os.path.join(output_directory, f'split_{os.path.splitext(os.path.basename(input_file))[0]}_{minute}.wav')
        split_audio.export(output_file, format='wav')
        print(f"Berhasil memisahkan audio {os.path.basename(input_file)} menit ke-{minute + 1}.")

    print("Proses pemisahan audio selesai.")

#input_file = input("Masukkan file audio: ")
split_audio_per_minute(input_file)

# 10. Final process (Run webUI)

In [ ]:
#@title Run Web
%cd /content/Retrieval-based-Voice-Conversion-WebUI
# %load_ext tensorboard
# %tensorboard --logdir /content/Retrieval-based-Voice-Conversion-WebUI/logs
!python3 infer-web.py --colab --pycmd python3

In [ ]:
#@title Copy Training Result to Drive (standart)
if mode == "training":
  !mkdir -p /content/drive/MyDrive/RVC/{MODELNAME}
  !mkdir -p /content/drive/MyDrive/RVC/weights

  !cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_*.pth /content/drive/MyDrive/RVC/{MODELNAME}
  !cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_*.pth /content/drive/MyDrive/RVC/{MODELNAME}
  !cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/added_*.index /content/drive/MyDrive/RVC/{MODELNAME}
  !cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/trained*.index /content/drive/MyDrive/RVC/{MODELNAME}
  !cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/total_*.npy /content/drive/MyDrive/RVC/{MODELNAME}

  !cp /content/Retrieval-based-Voice-Conversion-WebUI/weights/{MODELNAME}.pth /content/drive/MyDrive/RVC/weights/{MODELNAME}.pth
elif mode == "splitting":
  print("Mode set to splitting. Skipping this section")

# 11. copy training result (advanced)

In [ ]:
#@title Start copying
#@markdown Same with standart copy but you can input the name manualy
import ipywidgets as widgets
from IPython.display import display

modelname_input = widgets.Text(
    value='',
    placeholder='Masukkan nama model',
    description='Nama Model:',
    disabled=False
)

display(modelname_input)

copy_button = widgets.Button(description="Salin Hasil Pelatihan")

def copy_training_result(button):
    modelname = modelname_input.value.strip()
    if modelname:
        !mkdir -p /content/drive/MyDrive/RVC/{modelname}
        !mkdir -p /content/drive/MyDrive/RVC/weights

        !cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{modelname}/G_*.pth /content/drive/MyDrive/RVC/{modelname}
        !cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{modelname}/D_*.pth /content/drive/MyDrive/RVC/{modelname}
        !cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{modelname}/added_*.index /content/drive/MyDrive/RVC/{modelname}
        !cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{modelname}/trained*.index /content/drive/MyDrive/RVC/{modelname}
        !cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{modelname}/total_*.npy /content/drive/MyDrive/RVC/{modelname}

        !cp /content/Retrieval-based-Voice-Conversion-WebUI/weights/{modelname}.pth /content/drive/MyDrive/RVC/weights/{modelname}.pth
        print("Hasil pelatihan telah disalin ke Google Drive.")
    else:
        print("Nama model tidak boleh kosong.")

copy_button.on_click(copy_training_result)
display(copy_button)


# <center> Confused? Yes Me Too<br />
# <center> Some [☕](https://ko-fi.com/mocci24)?